In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
locations = ['Chão da Ribeira', 'Sexial', 'Funchal']

In [3]:
def contains_number(input_string):
    for char in input_string:
        if char.isdigit():
            return True
    return False

def parse_title(title): 
    parsed_title = {}

    # todo: optimize
    # todo: handle case when there are several locations in title
    # todo: move to separated fn
    for location in locations: 
        if location in title: 
            parsed_title['location'] = location
            break
        else:
            parsed_title['location'] = '-'

    title_arr = title.split('–')
    parsed_title["title"] = title_arr[0]
    # solve problem with different types of separators
    if len(title_arr) > 1: 
        for arr_elem in title_arr[1:]:
            if contains_number(arr_elem):
                parsed_title["date"] = arr_elem
            else:
                parsed_title["date"] = '-'
    else:
        parsed_title["date"] = '-'
    return parsed_title

In [4]:
url = 'https://www.madeira-web.com/en/whats-on/madeira-events.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')


In [5]:
fields = ["title", "date",  "location", ]

data = {field: [] for field in fields}

In [6]:
jan_h3 = soup.find('h3', id='jan')
feb_h3 = soup.find('h3', id='feb')

jan_to_feb_paragraphs = []

current_tag = jan_h3.find_next(['p', 'h3'])  
while current_tag and current_tag != feb_h3: 
    jan_to_feb_paragraphs.append(current_tag)
    current_tag = current_tag.find_next(['p', 'h3'])

for paragraph in jan_to_feb_paragraphs:
    title_data = parse_title(paragraph.find('strong').text)
    data['title'].append(title_data['title'])
    # todo: add month name to date 
    data['date'].append(title_data['date'])
    data['location'].append(title_data['location'])


In [7]:
df = pd.DataFrame(data)

df

,title,date,location
0,Madeira Plunge,January 1st,-
1,Madeira Orienteering Festival (MOF) 2025,"10-12, 2025",-
2,Madeira Orienteering Meeting (MOM) 2025,-,-
3,Funchal Marathon,-,Funchal
4,Poncha & Honey Exhibition (Mostra da Poncha e ...,-,-
5,"Panelo Festival - Chão da Ribeira, Seixal",-,Chão da Ribeira


2024-08-24 18:51:47,399 - INFO - Starting scraping
2024-08-24 18:51:47,732 - INFO - Fetched Madeira events
2024-08-24 18:51:50,076 - INFO - Fetched Eventbrite events
2024-08-24 18:51:50,188 - INFO - Parsed Madeira Events events
2024-08-24 18:51:50,244 - INFO - Parsed Eventbrite events
2024-08-24 18:51:50,550 - INFO - Scraped Madeira event: Waterline exhibition
2024-08-24 18:51:50,853 - INFO - Scraped Madeira event: Exhibit Siza & Oscar beyond the sea
2024-08-24 18:51:53,219 - INFO - Scraped Madeira event: Exhibit Siza & Oscar beyond the sea
2024-08-24 18:51:53,524 - INFO - Scraped Madeira event: L Concerts
2024-08-24 18:51:53,840 - INFO - Scraped Madeira event: Concerts ” Tuesdays´at the Fort”
2024-08-24 18:51:54,174 - INFO - Scraped Madeira event: 19th European Folklore Week
2024-08-24 18:51:54,685 - INFO - Scraped Madeira event: Wine Festival
2024-08-24 18:51:55,001 - INFO - Scraped Madeira event: Madeira Wine Lounge
2024-08-24 18:51:55,468 - INFO - Scraped Madeira event: Blandy´s Wi

,id,title,start_date,end_date,categories,location,address,description,short_description,source_link,source_id,image_link,organizer,price,last_inserted_at,last_updated_at
0,0,Waterline exhibition,2023-06-20 10:00:00,2028-06-20 18:00:00,Culture,Funchal,Museu de História Natural,The Natural History Museum of Funchal hosts t...,None,https://eventsmadeira.com/en/event/waterline-e...,1,https://eventsmadeira.com/oordypsa/2023/06/Mus...,None,None,2024-08-24 18:51:50.549479,2024-08-24 18:51:50.549484
1,1,Exhibit Siza & Oscar beyond the sea,2024-04-12 09:00:00,2024-09-13 17:30:00,Culture,Funchal,Fortaleza de São João Baptista do Pico,"The Fortress of São João Baptista do Pico, in...",None,https://eventsmadeira.com/en/event/exhibit-siz...,1,https://eventsmadeira.com/oordypsa/2024/04/FF-...,None,None,2024-08-24 18:51:50.851457,2024-08-24 18:51:50.851463
2,2,L Concerts,2024-07-27 22:00:00,2024-10-31 00:00:00,Music,Ponta do Sol,None,"Estalagem da Ponta do Sol, in collaboration w...",None,https://eventsmadeira.com/en/event/l-concerts/,1,https://eventsmadeira.com/oordypsa/2024/07/Est...,None,None,2024-08-24 18:51:53.523185,2024-08-24 18:51:53.523191
3,3,Concerts ” Tuesdays´at the Fort”,2024-08-06 20:30:00,2024-08-27 22:00:00,"Music, Culture",Porto Moniz,Forte de São João Baptista - Porto Moniz,The Municipality of Porto Moniz and Retoiça –...,None,https://eventsmadeira.com/en/event/concerts-tu...,1,https://eventsmadeira.com/oordypsa/2024/08/For...,None,None,2024-08-24 18:51:53.838599,2024-08-24 18:51:53.838603
4,4,19th European Folklore Week,2024-08-25 21:00:00,2024-08-29 23:00:00,Culture,Funchal,Auditório do Jardim Municipal,"Folklore festival, organized by the Boa Nova ...",None,https://eventsmadeira.com/en/event/17th-europe...,1,https://eventsmadeira.com/oordypsa/2022/08/Bot...,None,None,2024-08-24 18:51:54.172952,2024-08-24 18:51:54.172959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,209,Concertos L - ZDB convida Niecy Blues,2024-08-28 22:00:00,2024-08-28 23:30:00,Other,Estalagem da Ponta do Sol,"Quinta da Rocinha, 06 9360 -529 Ponta do Sol P...","<div class=""has-user-generated-content event-d...",None,https://www.eventbrite.com/e/concertos-l-zdb-c...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,Estalagem da Ponta do Sol,Free,2024-08-24 18:52:28.250666,2024-08-24 18:52:28.250667
10,210,bye bye summer party at NÔMADE,2024-08-31 22:30:00,2024-08-31 22:30:00,Holiday,NOMADE Restaurante,108 Rua Imperatriz Dona Amelia 9000-018 Funcha...,"<div class=""has-user-generated-content event-d...",None,https://www.eventbrite.pt/e/bye-bye-summer-par...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,NÔMADE fusion restaurante,Free,2024-08-24 18:52:29.764101,2024-08-24 18:52:29.764102
11,211,Community Meditation & Inspiration Sessions,2024-08-27 13:30:00,2024-08-27 13:30:00,Health,Sangha Cowork Funchal,98 Rua da Casa Branca 9004-535 Funchal Portugal,"<div class=""has-user-generated-content event-d...",None,https://www.eventbrite.pt/e/community-meditati...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,Sangha Cowork,Free,2024-08-24 18:52:31.562882,2024-08-24 18:52:31.562891
12,212,Free trial day,2024-08-26 21:30:00,2024-08-26 18:30:00,Community,Sangha Cowork Funchal,98 Rua da Casa Branca 9004-535 Funchal Portugal,"<div class=""has-user-generated-content event-d...",None,https://www.eventbrite.pt/e/free-trial-day-tic...,2,https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc....,Sangha Cowork,Free,2024-08-24 18:52:34.279387,2024-08-24 18:52:34.279390
